In [1]:
tissues_with_atac = ['CHO','Brain','Liver','Lung','Kidney','Spleen','BMDMwt','BMDM1hKLA']

In [2]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']


In [3]:
tss_annotation = doc['tss_annotation']

In [4]:
import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 15)


In [6]:
from ATAC_helper import *

## 1. Using final bed file vs the refseq genes CHO peaks and ATAC

In [10]:
ls Results/Figures/

cdf_enhancers_tissues_geneID_df_02.png  Distance_to_TSS_neg_included.png
cdf_enhancers_tissues.png               Distance_to_TSS.png
cdf_tissues/                            Figure2/
cdf_tissues_geneID_df_02.png            Figure3/
Distance_to_TSS_500.png                 SuppFigure4/


In [11]:
cmd = "computeMatrix reference-point -S /data/isshamie/TSS/1015_Processed/CHO/ATAC/out/signal/macs2/pooled_rep/CHO_ATAC1.trim.nodup.tn5_CHO_ATAC3.trim.nodup.tn5.pf.fc.signal.bigwig -R Results/output/TSS1.exp.bed -a 1000 -b 1000 -out Results/Figures/SuppFigure4/exp_all --numberOfProcessors 6"

In [13]:
cmd = "computeMatrix reference-point -S /data/isshamie/TSS/1015_Processed/CHO/ATAC/out/signal/macs2/pooled_rep/CHO_ATAC1.trim.nodup.tn5_CHO_ATAC3.trim.nodup.tn5.pf.fc.signal.bigwig -R Results/peak_groups/CHO/confident_promoters.bed -a 1000 -b 1000 -out Results/Figures/SuppFigure4/exp_all --numberOfProcessors 6"

In [14]:
!{cmd}

In [9]:
outfile = "Results/Figures/SuppFigure4/exp_all"
tss_bed = "Results/output/TSS1.exp.bed"
t = "CHO"

bigwig = glob.glob(os.path.join(data_folder,t, 'ATAC/out/signal/macs2/pooled_rep/*.fc.signal.bigwig'))[0]
run_heatmap(tss_bed, bigwig, outfile, dist=1000, numProc=6, dispImage=True)

computeMatrix reference-point -S /data/isshamie/TSS/1015_Processed/CHO/ATAC/out/signal/macs2/pooled_rep/CHO_ATAC1.trim.nodup.tn5_CHO_ATAC3.trim.nodup.tn5.pf.fc.signal.bigwig -R Results/output/TSS1.exp.bed -a 1000 -b 1000 -out Results/Figures/SuppFigure4/exp_all --numberOfProcessors 6 
plotHeatmap     -m Results/Figures/SuppFigure4/exp_all     -out Results/Figures/SuppFigure4/exp_all.png      --heatmapHeight 15       --refPointLabel TSS.center      --regionsLabel TSS      --plotTitle 'ATAC signal' 


IOError: [Errno 2] No such file or directory: u'Results/Figures/SuppFigure4/exp_all.png'

In [ ]:
#t = 'CHO'
for t in tissues_with_atac:
    print(t)
    if os.path.exists(os.path.join(data_folder,t, 'ATAC/out/signal/macs2/pooled_rep/')):
        bigwig = glob.glob(os.path.join(data_folder,t, 'ATAC/out/signal/macs2/pooled_rep/*.fc.signal.bigwig'))[0]
    else:
        bigwig = glob.glob(os.path.join(data_folder,t, 'ATAC/out/signal/macs2/rep1/*.fc.signal.bigwig'))[0]


    curr_in = (os.path.join('Results/peak_groups/', t))
    curr_out = (os.path.join('Results/ATAC_results/', t))
    if not os.path.exists(curr_out):
        os.mkdir(curr_out)

    tss_bed = os.path.join(curr_in, 'confident_promoters.bed')
    outfile = os.path.join(curr_out, 'confident_promoters')
    a = run_heatmap(tss_bed, bigwig, outfile, dist=1000, numProc=6, dispImage=True)
    display(a)

    tss_bed = os.path.join(curr_in, 'no_peaks.bed')
    outfile = os.path.join(curr_out, 'no_peaks')
    b = run_heatmap(tss_bed, bigwig, outfile, dist=1000, numProc=6, dispImage=True)
    display(b)

    tss_bed = os.path.join(curr_in, 'confident_enhancers.bed')
    outfile = os.path.join(curr_out, 'confident_enhancers')
    c = run_heatmap(tss_bed, bigwig, outfile, dist=1000, numProc=6, dispImage=True)
    display(c)


    tss_bed = os.path.join(curr_in, 'confident_promoters.bed')
    outfile = os.path.join(curr_out, 'confident_promoters_sorted_distance')
    d = run_heatmap(
        tss_bed,
        bigwig,
        outfile,
        dist=250,
        numProc=6,
        dispImage=True,
        computeMat_args='--sortRegions keep',
        plotHeat_args='--sortRegions keep')
    display(d)


    tss_bed = os.path.join(curr_in, 'confident_promoters_noDiv_PICR.bed')
    outfile = os.path.join(curr_out, 'confident_promoters_noDiv_PICR')
    e = run_heatmap(
        tss_bed,
        bigwig,
        outfile,
        dist=250,
        numProc=6,
        dispImage=True,
        computeMat_args='--sortRegions keep',
        plotHeat_args='--sortRegions keep')
    display(e)

    tss_bed = os.path.join(curr_in, 'confident_promoters_noDiv.bed')
    outfile = os.path.join(curr_out, 'confident_promoters_noDiv')
    f = run_heatmap(
        tss_bed,
        bigwig,
        outfile,
        dist=250,
        numProc=6,
        dispImage=True,
        computeMat_args='--sortRegions keep',
        plotHeat_args='--sortRegions keep')
    display(f)


## 2. All the peaks, use whichever had an ATAC if any

## 3. Peak 1 vs others

## Panel B
## silenced genes in refseq

## 3. Peak 1 vs others